<a href="https://colab.research.google.com/github/duckduck123go/Circuit-simulator/blob/main/circuit_simulator_laplace_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import numpy as np
from sympy import symbols, Matrix, simplify, inverse_laplace_transform
import matplotlib.pyplot as plt

In [ ]:
components = []  # List to store components and their connections
voltage_source = None  # Store the voltage source and its nodes

In [ ]:
# Laplace variable
s = symbols('s')

def add_component():
    """Add a component to the circuit."""
    try:
        comp_type = component_type_var.get()
        comp_value = float(component_value_var.get())
        node_start = int(start_node_var.get())
        node_end = int(end_node_var.get())

        if node_start < 1 or node_end < 1 or node_start == node_end:
            messagebox.showerror("Error", "Invalid node numbers.")
            return

        components.append((comp_type, comp_value, node_start, node_end))
        update_component_list()
    except ValueError:
        messagebox.showerror("Error", "Invalid component value.")

In [ ]:
def update_component_list():
    """Update the component listbox."""
    component_listbox.delete(0, tk.END)
    for i, (comp_type, comp_value, node_start, node_end) in enumerate(components):
        component_listbox.insert(
            tk.END,
            f"{i + 1}. {comp_type} = {comp_value}, Nodes {node_start} to {node_end}"
        )

In [ ]:
def set_voltage_source():
    """Set the voltage source."""
    try:
        positive_node = int(entry_positive_voltage_node.get())
        negative_node = int(entry_negative_voltage_node.get())
        voltage_value = float(entry_voltage_value.get())
        global voltage_source
        voltage_source = (positive_node, negative_node, voltage_value)
        messagebox.showinfo("Voltage Source", "Voltage source set.")
    except ValueError:
        messagebox.showerror("Error", "Invalid voltage source input.")

In [ ]:
def solve_laplace():
    """Solve the circuit using Laplace Transform-based nodal analysis."""
    try:
        if not voltage_source:
            raise ValueError("Voltage source not defined.")

        if not components:
            raise ValueError("No components added.")

        # Extract all unique nodes
        nodes = sorted(set(n for _, _, n1, n2 in components for n in (n1, n2)))
        node_count = len(nodes) - 1  # Exclude ground node (node 0)

        # Map nodes to indices
        node_map = {n: i for i, n in enumerate(nodes)}

        # Initialize admittance matrix and current vector
        Y = np.zeros((node_count, node_count), dtype=object)
        I = np.zeros(node_count, dtype=object)

        # Populate admittance matrix
        for comp_type, value, n1, n2 in components:
            n1_idx = node_map[n1] - 1 if n1 != 0 else None
            n2_idx = node_map[n2] - 1 if n2 != 0 else None

            # Calculate admittance
            if comp_type == "Resistor":
                admittance = 1 / value
            elif comp_type == "Inductor":
                admittance = 1 / (s * value)
            elif comp_type == "Capacitor":
                admittance = s * value
            else:
                raise ValueError(f"Unknown component type: {comp_type}")

            # Update admittance matrix
            if n1_idx is not None:
                Y[n1_idx, n1_idx] += admittance
                if n2_idx is not None:
                    Y[n1_idx, n2_idx] -= admittance
                    Y[n2_idx, n1_idx] -= admittance
            if n2_idx is not None:
                Y[n2_idx, n2_idx] += admittance

        # Add voltage source to the current vector
        pos_idx = node_map[voltage_source[0]] - 1
        neg_idx = node_map[voltage_source[1]] - 1
        voltage = voltage_source[2]

        if pos_idx >= 0:
            I[pos_idx] += voltage / s
        if neg_idx >= 0:
            I[neg_idx] -= voltage / s

        # Solve for node voltages in the Laplace domain
        Y_matrix = Matrix(Y)
        I_vector = Matrix(I)
        V_laplace = simplify(Y_matrix.inv() * I_vector)

        # Convert to time domain
        t = symbols('t', positive=True)
        V_time = [inverse_laplace_transform(v, s, t) for v in V_laplace]

        # Display results
        results = {nodes[i + 1]: (V_laplace[i], V_time[i]) for i in range(node_count)}
        for node, (v_lap, v_time) in results.items():
            print(f"Node {node}:")
            print(f"  Voltage (Laplace): {v_lap}")
            print(f"  Voltage (Time): {v_time}")

        # Plot results (example: node voltages over time)
        plt.figure(figsize=(10, 6))
        for i, v_time in enumerate(V_time, 1):
            v_func = lambdify(t, v_time, 'numpy')
            t_vals = np.linspace(0, 5, 1000)
            v_vals = v_func(t_vals)
            plt.plot(t_vals, v_vals, label=f"Node {nodes[i]} Voltage")

        plt.title("Node Voltages Over Time")
        plt.xlabel("Time (s)")
        plt.ylabel("Voltage (V)")
        plt.grid()
        plt.legend()
        plt.show()

    except Exception as e:
        messagebox.showerror("Error", str(e))

In [ ]:
# GUI Setup (unchanged from your original code)
root = tk.Tk()
root.title("RLC Circuit Simulator")

tk.Label(root, text="Component Type:").grid(row=0, column=0, padx=5, pady=5)
component_type_var = tk.StringVar(value="Resistor")
component_type_menu = ttk.Combobox(root, textvariable=component_type_var)
component_type_menu["values"] = ("Resistor", "Inductor", "Capacitor")
component_type_menu.grid(row=0, column=1, padx=5, pady=5)

tk.Label(root, text="Value:").grid(row=1, column=0, padx=5, pady=5)
component_value_var = tk.StringVar()
tk.Entry(root, textvariable=component_value_var).grid(row=1, column=1, padx=5, pady=5)

tk.Label(root, text="Start Node:").grid(row=2, column=0, padx=5, pady=5)
start_node_var = tk.StringVar(value="1")
tk.Entry(root, textvariable=start_node_var).grid(row=2, column=1, padx=5, pady=5)

tk.Label(root, text="End Node:").grid(row=3, column=0, padx=5, pady=5)
end_node_var = tk.StringVar(value="2")
tk.Entry(root, textvariable=end_node_var).grid(row=3, column=1, padx=5, pady=5)

tk.Button(root, text="Add Component", command=add_component).grid(row=4, column=0, columnspan=2, pady=5)

component_listbox = tk.Listbox(root, height=10, width=50)
component_listbox.grid(row=5, column=0, columnspan=2, pady=5)

tk.Label(root, text="Voltage Source: Positive Node").grid(row=6, column=0, padx=5, pady=5)
entry_positive_voltage_node = tk.Entry(root)
entry_positive_voltage_node.grid(row=6, column=1, padx=5, pady=5)

tk.Label(root, text="Negative Node").grid(row=7, column=0, padx=5, pady=5)
entry_negative_voltage_node = tk.Entry(root)
entry_negative_voltage_node.grid(row=7, column=1, padx=5, pady=5)

tk.Label(root, text="Voltage Value (V)").grid(row=8, column=0, padx=5, pady=5)
entry_voltage_value = tk.Entry(root)
entry_voltage_value.grid(row=8, column=1, padx=5, pady=5)

tk.Button(root, text="Set Voltage Source", command=set_voltage_source).grid(row=9, column=0, columnspan=2, pady=5)

tk.Button(root, text="Solve Circuit", command=solve_laplace).grid(row=10, column=0, columnspan=2, pady=10)

root.mainloop()